## Importing The required Libraries

In [ ]:
import pandas as pd

import chess
import chess.pgn

import time

###  List of Raw data files considered

In [ ]:
PGNfilNames = ['Modern.pgn','Reti2b3.pgn','Caro-Kann2c4.pgn','Caro-KannAdv.pgn','Caro-KannClassic.pgn','Caro-KannEx.pgn','FourKnights.pgn','GiuocoPiano.pgn','KingsGambit.pgn','Nimzowitsch-Larsen.pgn','QID4e3.pgn','RuyLopezMarshall.pgn','ScotchGambit.pgn','SicilianDragonOther6.pgn','SicilianRossolimo.pgn']

#### PGN Reader

In [ ]:
def PGNtoDF(PGNfilNames):
    """
    The input is list of pgn file names with dot extensions.
    Must be a list of strings even if there is only one element
    """
    AllGames = []
    for Nam in PGNfilNames:
        #Open PGN as Stream
        print('Opened',Nam)
        PGNFile = open(Nam)
        
        TestGame = chess.pgn.read_game(PGNFile)
        Moves = TestGame.mainline()
        
        Game = PGNFile.read()
        TotalEvents = Game.count('[Event')
        
        PGNFile.seek(0,0)
        
        
        for i in range(0,TotalEvents):
            #Gam = ['modern']
            G = chess.pgn.read_game(PGNFile)
            Gam = list(G.headers.values())
            
            if len(Gam)< 10:
                for i in range (0,10-len(Gam)):
                    Gam.append('')
            
            for j in range(0,len(Gam)):
                if '?' in Gam[j]:
                    Gam[j]=Gam[j].replace('?','0')
                elif Gam[j]=='':
                    Gam[j] = 0
            
            Gam.append(Nam[:-4])
            
            #Moves PGN to spaced string cleaner starts
            Moves = str(G.mainline())
            SetLen = Moves.count('.')
            for j in range(0,Moves.count('.')+1):
                Remaining = Moves.count('.')
                Moves = Moves.replace(' '+str(j)+'.',' ')
                if Moves.count('.') == Remaining:
                    Moves = Moves.replace(str(j)+'.',' ')
                
                Moves=Moves.replace('  ',' ')
              
            for j in range(0,SetLen):
                Moves = Moves.replace(' '+str(j)+' ',' ')
            
            #Moves PGN to spaced string cleaner ends
            
            Gam.append(Moves)
            Gam.append(SetLen)
        
            AllGames.append(Gam)
     
    MainDF = pd.DataFrame(AllGames,columns = ['Event', 'Site', 'Date','Round','White','Black','Result','BlackElo','ECO','WhiteElo','Opening','Moves','SetLen'])
    MainDF['Result']=MainDF['Result'].apply(lambda X: 'W' if X=='1-0' else 'B' if X=='0-1' else 'D')
    MainDF['Moves'] = MainDF['Moves'].apply(lambda X : '' if not type(X)==str else X)
    MainDF['Set1'] = MainDF['Moves'].apply(lambda X: X[0:10].split())
    MainDF['W1'] = MainDF['Set1'].apply(lambda X: X[0] if len(X)>0 else '')
    MainDF['B1'] = MainDF['Set1'].apply(lambda X: X[1] if len(X)>0 else '')
    return MainDF